In [1]:
import os

go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
os.chdir(go_up_n_directories(os.getcwd(), 4))  # run once (otherwise restart kernel)
os.getcwd()

'/home/tim/Development/OCPPM'

In [11]:
# Python natives
import pprint
import numpy as np
import pandas as pd
import pickle
import time
import json
import os
import re
from utilities import evaluation_utils
from ocpa.algo.predictive_monitoring.obj import Feature_Storage
from ocpa.algo.predictive_monitoring import tabular

In [3]:
feature_storage_file = "data/BPI17/feature_encodings/EFG/efg/raw/BPI_split_[C2_P2_P3_P5_O3_Action_EventOrigin_OrgResource].fs"
target = ("event_remaining_time", ())
model_output_path = "models/BPI17/baselines/median"

In [4]:
with open(feature_storage_file, "rb") as bin:
    feature_storage: Feature_Storage = pickle.load(bin)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# flatten EFG with same train/test split
eft_train = tabular.construct_table(
    feature_storage, feature_storage.train_indices + feature_storage.validation_indices
)
eft_test = tabular.construct_table(feature_storage, feature_storage.test_indices)

In [6]:
# subset relevant data
y_train = eft_train[target]
y_test = eft_test[target]

In [7]:
train_start_time = time.time()
# fit the model ;)
median = y_train.median()
total_train_time = time.time() - train_start_time

In [8]:
pred_start_time = time.time()
y_train_preds = [median] * y_train.shape[0]
train_pred_time = time.time() - pred_start_time

pred_start_time = time.time()
y_test_preds = [median] * y_test.shape[0]
test_pred_time = time.time() - pred_start_time

In [9]:
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, regression=True, time=train_pred_time
)
eval_train["report"]["training_time"] = total_train_time
eval_valid = evaluation_utils.get_evaluation(
    y_test, y_test_preds, regression=True, time=test_pred_time
)
evaluation_report = {"Train": eval_train, "Test": eval_valid}

In [12]:
with open(os.path.join(model_output_path, "evaluation_report.json"), "w") as fp:
    json.dump(evaluation_report, fp, indent=2)

pprint.pprint(evaluation_report)

{'Test': {'report': {'MAE': 0.7746246264487259,
                     'MAPE': 4.717113662945489,
                     'MSE': 1.047181432735379,
                     'R^2': -0.06980023131687063,
                     'prediction_time': 0.00015997886657714844}},
 'Train': {'report': {'MAE': 0.7854286288089017,
                      'MAPE': 4.976396720117326,
                      'MSE': 1.0802058186052832,
                      'R^2': -0.07491865804165632,
                      'prediction_time': 0.00029540061950683594,
                      'training_time': 0.004192829132080078}}}
